# Данные

In [1]:
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

Читаем мета-файл (дважды, потому что один из них будем менять, а второй референсить). Можно было бы сделать более логично, но я почему-то решила так.

In [2]:
with open('../preprocess/meta.json', 'r') as f:
    meta = json.load(f)

with open('../preprocess/meta.json', 'r') as f:
    original_meta = json.load(f)

Загружаем файл с обработанными данными, заменяем все пустые ячейки на `none`. Откуда пустые ячейки? Они возникают в вариантах препроцессинга, где используется удаление стоп-слов и/или удаление эмоджи (например, когда комментарий изначально состоял только из эмоджи)

In [3]:
def load_data(filepath):
    df = pd.read_csv(filepath, index_col=0, sep='\t')
    print(f'loaded from {filepath}')
    print(df['разметка'].value_counts())
    return df

In [4]:
data = load_data('../Data/data_preprocessed.tsv')
data.fillna('none',  inplace=True)
assert data.isnull().values.any() == False

loaded from ../Data/data_preprocessed.tsv
negative    491
positive    410
neutral     357
speech      213
Name: разметка, dtype: int64


Функция для кодирования разметки в числовые классы. **NB**: классы `neutral` и `speech` объединяются (чтобы было больше данных)

In [5]:
def encode_labeling(label):
    if label == 'negative':
        label_id = -1
    elif label == 'neutral':
        label_id = 0
    elif label == 'positive':
        label_id = 1
    elif label == 'speech':
        label_id = 0
        
    return label_id

Самая обычная tf-idf векторизация, т.к. данных мало и объем вокабуляра не превышает 5к слов. Для ускорения работы используется `analyzer=str.split`, чтобы отключить встроенную токенизацию (т.к. все наши данные и так токенизированны через пробел)

In [6]:
def transform_data(data, col_name):
    y = data['разметка'].apply(encode_labeling)
    
    corpus = data[col_name]
    
    vectorizer = TfidfVectorizer(analyzer=str.split)
    X = vectorizer.fit_transform(corpus)
    
    return X, y

In [7]:
prep_id = str(0)
X, y = transform_data(data, prep_id)
print(X.shape)
original_meta[prep_id]

(1471, 4821)


{'punctuation_deletion': 'no',
 'ner_processing': 'no',
 'lemmatization': 'no',
 'stopwords_deletion': 'no',
 'emojis_processing': 'no',
 'vulgar_processing': 'no'}

# Модели

Мы решили сравнить 5 алгоритмов класификации

In [21]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier


SEED = 67

In [22]:
def fit_pred(X_train, X_test, y_train, y_test, clf=None):
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    print_report = classification_report(y_test, pred)
    report = classification_report(y_test, pred, output_dict=True)
    #print(print_report)
    return pred, print_report, report

Гридсерч используется ниже, уже после определения наиболее удачной модели и препроцессинга

In [23]:
def grid_pred(X_train, X_test, y_train, y_test, clf=None, params={}, cv=5):
    grid = GridSearchCV(clf, params, cv=cv)
    grid.fit(X_train, y_train)
    #print(grid.best_params_)
    
    best = grid.best_estimator_
    
    best.fit(X_train, y_train)
    pred = best.predict(X_test)
    print_report = classification_report(y_test, pred)
    report = classification_report(y_test, pred, output_dict=True)
    #print(print_report)
    
    return best, pred, print_report, report

Сама функция которая обучает все модели и возвращает отчет с метриками

In [24]:
def try_models(X_train, X_test, y_train, y_test, log=False):
    
    clf_logreg = LogisticRegression(random_state=SEED, solver='liblinear')
    clf_sgd = SGDClassifier(random_state=SEED)
    clf_forest = RandomForestClassifier(random_state=SEED)
    clf_bayes = MultinomialNB()
    clf_xgboost = XGBClassifier()

    clfs = {'logreg': {'model': clf_logreg, 'report': {}},
            'sgd': {'model': clf_sgd, 'report': {}},
            'forest': {'model': clf_forest, 'report': {}}, 
            'bayes':  {'model': clf_bayes,  'report': {}},
            'xgboost':  {'model': clf_xgboost,  'report': {}}
           }
    
    for name, clf in clfs.items():
        model = clf['model']
        _, print_report, report = fit_pred(X_train, X_test, y_train, y_test, clf=model)
        clfs[name]['report'] = report
        
        
        if log:
            print(name.upper())
            print(print_report)
        
    return clfs

Пример использования на данных, заданных выше

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED, stratify=y)
report = try_models(X_train, X_test, y_train, y_test, log=True)

LOGREG
              precision    recall  f1-score   support

          -1       0.79      0.65      0.71       148
           0       0.65      0.87      0.74       171
           1       0.90      0.66      0.76       123

    accuracy                           0.74       442
   macro avg       0.78      0.73      0.74       442
weighted avg       0.77      0.74      0.74       442

SGD
              precision    recall  f1-score   support

          -1       0.76      0.68      0.72       148
           0       0.66      0.78      0.72       171
           1       0.83      0.73      0.78       123

    accuracy                           0.73       442
   macro avg       0.75      0.73      0.74       442
weighted avg       0.74      0.73      0.73       442

FOREST
              precision    recall  f1-score   support

          -1       0.79      0.61      0.69       148
           0       0.62      0.85      0.72       171
           1       0.84      0.62      0.71       123

  

In [26]:
print('weighted avg f1-score\n')

for name, clf in report.items():
    print(f"{name.upper()}\t{clf['report']['weighted avg']['f1-score']}")
          
original_meta[prep_id]

weighted avg f1-score

LOGREG	0.7374347908073318
SGD	0.7340997854448016
FOREST	0.7071838275544589
BAYES	0.7410735405762653
XGBOOST	0.6822678301646522


{'punctuation_deletion': 'no',
 'ner_processing': 'no',
 'lemmatization': 'no',
 'stopwords_deletion': 'no',
 'emojis_processing': 'no',
 'vulgar_processing': 'no'}

## Исследование моделей с базовыми параметрами и всех препроцессингов

In [27]:
import numpy as np
from tqdm.auto import tqdm

Здесь обучается и замеряется кач-во 960 моделей (за ~8 минут)

In [28]:
for m_id, m in tqdm(meta.items()):
    X, y = transform_data(data, m_id)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED, stratify=y)
    
    reports = try_models(X_train, X_test, y_train, y_test)
    
    meta[m_id]['reports'] = reports

In [ ]:
# если есть файл, мжно подгрузить его

with open('compare_meta.pickle', 'rb') as f:
    meta_from_file = pickle.load(f)
    
# meta = meta_from_file

В качестве метрики выбрали weighted average f1-score, вырезаем только его, чтобы можно было внимательнее посмотреть на результаты

In [29]:
weighted_fs = []
preps = []
models = []

for m_id, m in meta.items():
    #print(original_meta[m_id])
    
    for name, clf in m['reports'].items():
        score = clf['report']['weighted avg']['f1-score']
        
        weighted_fs.append(score)
        preps.append(m_id)
        models.append(name)
        #print(f"{name.upper()}\t{score}")
        
    #print('\n===========\n')

Все варианты с метрикой выше `0.75`:

In [31]:
top_prep = []
top_model = []

for i in np.argsort(weighted_fs)[::-1]:
    if weighted_fs[i] < 0.75:
        break
    top_prep.append(preps[i])
    top_model.append(models[i])
    print(f'{preps[i]}\t{models[i]}\t{weighted_fs[i]}')

81	sgd	0.779170661715591
86	sgd	0.7720163909018399
55	logreg	0.7717005314453232
87	logreg	0.7717005314453232
23	logreg	0.7717005314453232
55	sgd	0.7706759599004663
85	logreg	0.7691577909208065
20	sgd	0.7690593212382701
87	sgd	0.7682864093475529
52	sgd	0.7672515969379039
48	sgd	0.7669885503015509
21	logreg	0.7668862890449263
53	logreg	0.7668862890449263
51	logreg	0.7665839819182566
19	logreg	0.7665839819182566
83	logreg	0.7665839819182566
176	sgd	0.76604592866399
22	sgd	0.7655973947668203
55	forest	0.7652950087029121
117	logreg	0.7645595425092476
181	logreg	0.7645595425092476
149	logreg	0.7645595425092476
81	logreg	0.7641877753760317
49	logreg	0.7641877753760317
17	logreg	0.7641877753760317
85	sgd	0.7635909031733173
149	bayes	0.7633686569128357
181	bayes	0.7633686569128357
117	bayes	0.7633686569128357
49	sgd	0.7630524487419506
119	logreg	0.7625550125498421
151	logreg	0.7625550125498421
183	logreg	0.7625550125498421
80	sgd	0.7621709242482197
54	logreg	0.7620804534831686
86	logreg	0.76208

Разброс значений для наилучшего алгоритма:

In [32]:
for i in np.argsort(weighted_fs)[::-1]:
    if models[i] == 'sgd':
        print(f'{preps[i]}\t{models[i]}\t{weighted_fs[i]}')

81	sgd	0.779170661715591
86	sgd	0.7720163909018399
55	sgd	0.7706759599004663
20	sgd	0.7690593212382701
87	sgd	0.7682864093475529
52	sgd	0.7672515969379039
48	sgd	0.7669885503015509
176	sgd	0.76604592866399
22	sgd	0.7655973947668203
85	sgd	0.7635909031733173
49	sgd	0.7630524487419506
80	sgd	0.7621709242482197
54	sgd	0.7612164217330719
84	sgd	0.7608654391132358
21	sgd	0.7588730736961566
17	sgd	0.7583581909451448
118	sgd	0.7573576806093847
23	sgd	0.7569036018838555
53	sgd	0.756849550561315
114	sgd	0.7561136113855315
16	sgd	0.7548980558049776
37	sgd	0.7535338382844777
146	sgd	0.7533458915808308
83	sgd	0.7523905065582697
148	sgd	0.7500226407520483
181	sgd	0.750021728395085
112	sgd	0.7496177819845157
57	sgd	0.7494096383871373
125	sgd	0.7476702711581116
88	sgd	0.7476496590105673
61	sgd	0.747513379537516
177	sgd	0.7474977189292923
144	sgd	0.7472289075662439
178	sgd	0.7464083205702481
182	sgd	0.7463898403753455
69	sgd	0.7461486258199062
117	sgd	0.745793555410885
116	sgd	0.7457820196309313
149	s

Можно заметить, что лучший результат дает препроцессинг с заменой именованных сущностей, заменой мата на плейсходер и лемматизацией:

In [33]:
original_meta['81']

{'punctuation_deletion': 'no',
 'ner_processing': 'replace',
 'lemmatization': 'yes',
 'stopwords_deletion': 'no',
 'emojis_processing': 'no',
 'vulgar_processing': 'yes'}

А худший результат получается при удалении всего, что можно, и без нормализации (нет ни лемматизации, ни замены мата):

In [201]:
original_meta['138']

{'punctuation_deletion': 'yes',
 'ner_processing': 'del',
 'lemmatization': 'no',
 'stopwords_deletion': 'yes',
 'emojis_processing': 'del',
 'vulgar_processing': 'no'}

Посмотрим внимательнее на варианты с результатами `> 0.73`

In [34]:
from collections import Counter

In [35]:
c_models = Counter(top_model).most_common()
c_models

[('logreg', 50), ('sgd', 26), ('bayes', 9), ('forest', 4)]

In [36]:
[(n, f) for n, f in Counter(top_prep).most_common() if f > 1]

[('55', 3),
 ('85', 3),
 ('83', 3),
 ('181', 3),
 ('81', 2),
 ('86', 2),
 ('87', 2),
 ('23', 2),
 ('20', 2),
 ('52', 2),
 ('48', 2),
 ('21', 2),
 ('53', 2),
 ('22', 2),
 ('117', 2),
 ('149', 2),
 ('49', 2),
 ('17', 2),
 ('119', 2),
 ('151', 2),
 ('183', 2),
 ('80', 2),
 ('54', 2),
 ('84', 2),
 ('37', 2),
 ('118', 2),
 ('148', 2),
 ('16', 2)]

In [37]:
original_meta['83']

{'punctuation_deletion': 'no',
 'ner_processing': 'replace',
 'lemmatization': 'yes',
 'stopwords_deletion': 'no',
 'emojis_processing': 'del',
 'vulgar_processing': 'yes'}

In [38]:
original_meta['55']

{'punctuation_deletion': 'no',
 'ner_processing': 'del',
 'lemmatization': 'yes',
 'stopwords_deletion': 'no',
 'emojis_processing': 'label',
 'vulgar_processing': 'yes'}

In [39]:
original_meta['85']

{'punctuation_deletion': 'no',
 'ner_processing': 'replace',
 'lemmatization': 'yes',
 'stopwords_deletion': 'no',
 'emojis_processing': 'replace',
 'vulgar_processing': 'yes'}

Для дальнейшего изучения на всякий случай сохраняем все результаты в файл

In [40]:
json_meta = {}

for m_id, m in meta.items():
    json_meta[m_id] = {'prep': {},
                       'reports': {'logreg': {},
                                   'sgd': {},
                                   'forest': {}, 
                                   'bayes':  {},
                                   'xgboost':  {}}
                      }

for m_id, m in meta.items():
    json_meta[m_id]['prep'] = original_meta[m_id]
    for name, clf in m['reports'].items():
        json_meta[m_id]['reports'][name] = clf['report']

with open('compare.json', 'w') as f:
    json.dump(json_meta, f, ensure_ascii=False, indent=4)

Пример как выглядит первый из 192 элементов:

In [41]:
json_meta['0']

{'prep': {'punctuation_deletion': 'no',
  'ner_processing': 'no',
  'lemmatization': 'no',
  'stopwords_deletion': 'no',
  'emojis_processing': 'no',
  'vulgar_processing': 'no'},
 'reports': {'logreg': {'-1': {'precision': 0.7843137254901961,
    'recall': 0.5405405405405406,
    'f1-score': 0.6399999999999999,
    'support': 148},
   '0': {'precision': 0.6419213973799127,
    'recall': 0.8596491228070176,
    'f1-score': 0.735,
    'support': 171},
   '1': {'precision': 0.8018018018018018,
    'recall': 0.7235772357723578,
    'f1-score': 0.7606837606837608,
    'support': 123},
   'accuracy': 0.7149321266968326,
   'macro avg': {'precision': 0.742678974890637,
    'recall': 0.7079222997066387,
    'f1-score': 0.7118945868945868,
    'support': 442},
   'weighted avg': {'precision': 0.7340918822310762,
    'recall': 0.7149321266968326,
    'f1-score': 0.710337336117879,
    'support': 442}},
  'sgd': {'-1': {'precision': 0.7096774193548387,
    'recall': 0.5945945945945946,
    'f1-s

На еще более всякий случай сохраняю всю мету вместе с моделями

In [42]:
import pickle

In [43]:
with open('compare_meta.pickle', 'wb') as f:
    pickle.dump(meta, f)

Пример как выглядит первый из 192 элементов:

In [44]:
meta['0']

{'punctuation_deletion': 'no',
 'ner_processing': 'no',
 'lemmatization': 'no',
 'stopwords_deletion': 'no',
 'emojis_processing': 'no',
 'vulgar_processing': 'no',
 'reports': {'logreg': {'model': LogisticRegression(random_state=67, solver='liblinear'),
   'report': {'-1': {'precision': 0.7843137254901961,
     'recall': 0.5405405405405406,
     'f1-score': 0.6399999999999999,
     'support': 148},
    '0': {'precision': 0.6419213973799127,
     'recall': 0.8596491228070176,
     'f1-score': 0.735,
     'support': 171},
    '1': {'precision': 0.8018018018018018,
     'recall': 0.7235772357723578,
     'f1-score': 0.7606837606837608,
     'support': 123},
    'accuracy': 0.7149321266968326,
    'macro avg': {'precision': 0.742678974890637,
     'recall': 0.7079222997066387,
     'f1-score': 0.7118945868945868,
     'support': 442},
    'weighted avg': {'precision': 0.7340918822310762,
     'recall': 0.7149321266968326,
     'f1-score': 0.710337336117879,
     'support': 442}}},
  'sgd'

### Подбор параметров

В кач-ве данных возьмем препроцессинги, показавшие наилучший результат для каждой из 3 моделей с наилучшим скором

In [46]:
for m in c_models:
    for i in np.argsort(weighted_fs)[::-1]:
        if models[i] == m[0]:
            print(f'{preps[i]}\t{models[i]}\t{weighted_fs[i]}')
            break

55	logreg	0.7717005314453232
81	sgd	0.779170661715591
149	bayes	0.7633686569128357
55	forest	0.7652950087029121


In [47]:
grid_sgd = SGDClassifier(random_state=SEED, max_iter=10000)
params_sgd = {'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron', 
                       'squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
              'penalty': ['l1','l2', 'elasticnet'],
              'fit_intercept': [True, False],
              #'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive']
             }

grid_logreg = LogisticRegression(random_state=SEED, max_iter=200)
params_logreg =  {#'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 
                  'C': [0.001, 0.01, 0.1, 1, 10, 50, 100, 1000],
                  'fit_intercept': [True, False],
                 }

grid_forest = RandomForestClassifier(random_state=SEED)
params_forest = {'n_estimators': [10, 50, 100, 200], 
                 'max_depth': [2, 5, 10, 20, 50, 100],
                 'criterion': ['gini', 'entropy'],
                }

grid_pipeline = {
    'sgd': {'model': grid_sgd, 'grid': params_sgd, 'prep': '81',
            'best_model': None, 'best_params': {}, 'report': {}},
    
    'logreg': {'model': grid_logreg, 'grid': params_logreg, 'prep': '55',
               'best_model': None, 'best_params': {}, 'report': {}},
    
    'forest': {'model': grid_forest, 'grid': params_forest, 'prep': '55',
               'best_model': None, 'best_params': {}, 'report': {}},
}

In [49]:
%%time

for name, clf in grid_pipeline.items():
    
    X, y = transform_data(data, clf['prep'])
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED, stratify=y)
    
    best, pred, print_report, report = grid_pred(X_train, X_test, y_train, y_test, 
                                                 clf=clf['model'], params=clf['grid'], cv=3)
    
    best_params = best.get_params()
    grid_pipeline[name]['best_model'] = best
    grid_pipeline[name]['best_params'] = best_params
    grid_pipeline[name]['report'] = report
    
    print(f"{name} weighted avg f1 = {report['weighted avg']['f1-score']}")
    for param, value in best_params.items():
        if param in clf['grid'].keys():
            print(f'{param}={value}')
    print(print_report)

C:\Users\User\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_stochastic_gradient.py:573: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


sgd weighted avg f1 = 0.7626119813231003
fit_intercept=True
loss=hinge
penalty=l1
              precision    recall  f1-score   support

          -1       0.75      0.73      0.74       148
           0       0.73      0.80      0.77       171
           1       0.83      0.75      0.79       123

    accuracy                           0.76       442
   macro avg       0.77      0.76      0.76       442
weighted avg       0.77      0.76      0.76       442

logreg weighted avg f1 = 0.7632298414016918
C=1
fit_intercept=False
              precision    recall  f1-score   support

          -1       0.75      0.72      0.74       148
           0       0.71      0.82      0.76       171
           1       0.89      0.72      0.80       123

    accuracy                           0.76       442
   macro avg       0.78      0.76      0.77       442
weighted avg       0.77      0.76      0.76       442

forest weighted avg f1 = 0.7557183591521924
criterion=gini
max_depth=20
n_estimators=200

Попробуем еще поэкспериментировать с SGD:

In [51]:
%%time

grid_sgd = SGDClassifier(random_state=SEED, max_iter=100000)

params_sgd = {'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron', 
                       'squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
              'penalty': ['l1','l2', 'elasticnet'],
              'fit_intercept': [True, False],
              'class_weight': [None, 'balanced'],
             }

for name, clf in grid_pipeline.items():
    
    if name == 'sgd':
    
        X, y = transform_data(data, clf['prep'])

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED, stratify=y)

        best, pred, print_report, report = grid_pred(X_train, X_test, y_train, y_test, 
                                                     clf=grid_sgd, params=params_sgd, cv=2)

        best_params = best.get_params()
        grid_pipeline[name]['best_model'] = best
        grid_pipeline[name]['best_params'] = best_params
        grid_pipeline[name]['report'] = report

        print(name)
        for param, value in best_params.items():
            if param in clf['grid'].keys():
                print(f'{param}={value}')
        print(print_report)
        print(report['weighted avg']['f1-score'])

C:\Users\User\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_stochastic_gradient.py:573: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


sgd
fit_intercept=True
loss=modified_huber
penalty=l2
              precision    recall  f1-score   support

          -1       0.80      0.66      0.73       148
           0       0.71      0.82      0.76       171
           1       0.80      0.79      0.79       123

    accuracy                           0.76       442
   macro avg       0.77      0.76      0.76       442
weighted avg       0.77      0.76      0.76       442

0.7590857064109781
Wall time: 3min 6s


Не получилось, тут результаты все равно хуже чем с дефолтными параметрами, потому что даже на таком max_iter он не может зафититься...

Лучшей моделью получается:

In [52]:
best_sgd = SGDClassifier(random_state=SEED)
X, y = transform_data(data, '81')   
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED, stratify=y)
_, print_report, report = fit_pred(X_train, X_test, y_train, y_test, clf=best_sgd)
print(print_report)

print(report['weighted avg']['f1-score'])

              precision    recall  f1-score   support

          -1       0.74      0.76      0.75       148
           0       0.75      0.79      0.77       171
           1       0.87      0.78      0.82       123

    accuracy                           0.78       442
   macro avg       0.79      0.78      0.78       442
weighted avg       0.78      0.78      0.78       442

0.779170661715591


проверим влияние выбранного random_state

1. на модель:

In [56]:
seeds = []

for s in range(100):
    sgd = SGDClassifier(random_state=s)
    X, y = transform_data(data, '81')   
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED, stratify=y)
    _, print_report, report = fit_pred(X_train, X_test, y_train, y_test, clf=sgd)

    seeds.append((report['weighted avg']['f1-score']))
    
print(max(seeds), np.argmax(seeds))
print(np.mean(seeds))
print(min(seeds))

0.779170661715591 67
0.761831913258592
0.735995228156152


2. на деление тест-трейн:

In [57]:
seeds = []

for s in range(100):
    sgd = SGDClassifier(random_state=SEED)
    X, y = transform_data(data, '81')   
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=s, stratify=y)
    _, print_report, report = fit_pred(X_train, X_test, y_train, y_test, clf=sgd)

    seeds.append((report['weighted avg']['f1-score']))
    
print(max(seeds), np.argmax(seeds))
print(np.mean(seeds))
print(min(seeds))

0.7876388345021732 94
0.7381769095711054
0.6793165866635282


*Анна Полянская, 2020*